In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import os

# Ruta absoluta del directorio de datos del usuario
user_data_dir = os.path.abspath("chrome_user_data")

# Crear el directorio si no existe
if not os.path.exists(user_data_dir):
    os.makedirs(user_data_dir)

# Configurar opciones de Chrome
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Ejecutar en modo headless
chrome_options.add_argument("--start-maximized")  # Abrir el navegador en pantalla completa
# Establecer el perfil de usuario
chrome_options.add_argument(f"user-data-dir={user_data_dir}")

# Descargar y configurar el controlador de Chrome automáticamente
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Abrir el navegador y navegar a Apollo.io
driver.get("https://app.apollo.io/#/login")

# Esperar unos segundos para ver el navegador abierto antes de empezar a interactuar
time.sleep(5)

# Intentar encontrar el campo de correo electrónico por su XPath y rellenarlo
email_xpath = '/html/body/div[2]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/form/div[2]/div/div/input'
try:
    email_input = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, email_xpath))
    )
    email_input.send_keys("Gbustosgarcia01@gmail.com")
    print("El campo de correo electrónico fue rellenado.")
except Exception as e:
    print(f"El campo de correo electrónico no fue encontrado: {e}")

# Intentar encontrar el campo de contraseña por su XPath y rellenarlo
password_xpath = '/html/body/div[2]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/form/div[3]/div/div[1]/div/input'
try:
    password_input = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, password_xpath))
    )
    password_input.send_keys("LG030920..lg.")
    print("El campo de contraseña fue rellenado.")
except Exception as e:
    print(f"El campo de contraseña no fue encontrado: {e}")

# Intentar encontrar el botón de Log In por su XPath y hacer clic en él
login_button_xpath = '/html/body/div[2]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/form/div[4]/button'
try:
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, login_button_xpath))
    )
    login_button.click()
    print("El botón de Log In fue clickeado.")
except Exception as e:
    print(f"El botón de Log In no fue encontrado o no fue clickeable: {e}")

# Esperar unos segundos para asegurar que el inicio de sesión se complete
time.sleep(10)

# Cerrar el navegador
driver.quit()


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import csv
import time

# Configurar opciones de Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar en modo headless
chrome_options.add_argument("--start-maximized")  # Abrir el navegador en pantalla completa
user_data_dir = "chrome_user_data"
chrome_options.add_argument(f"user-data-dir={user_data_dir}")

# Descargar y configurar el controlador de Chrome automáticamente
def start_driver():
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)

driver = start_driver()

csv_file_path = 'Resultados/Empresas/Apollo_Acounts_1-10.csv'
rows = []
company_urls = {}  # Diccionario para mapear URLs a sus índices de fila

# Verificar y añadir columnas si no existen
with open(csv_file_path, mode='r', newline='', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)
    existing_fieldnames = csv_reader.fieldnames
    required_columns = ['Website', 'LinkeIn', 'Faceboock', 'Twitter', 'Descripción', 'Processed']
    
    if not all(column in existing_fieldnames for column in required_columns):
        fieldnames = existing_fieldnames + [column for column in required_columns if column not in existing_fieldnames]
    else:
        fieldnames = existing_fieldnames

    for index, row in enumerate(csv_reader):
        rows.append(row)
        company_urls[row.get('Company URL')] = index  # Guardar el índice de fila por URL

num_rows_to_process = 10000  # Cambia este valor según sea necesario
processed_rows = 0

def write_to_csv(rows, fieldnames):
    with open(csv_file_path, mode='w', newline='', encoding='utf-8') as updated_file:
        csv_writer = csv.DictWriter(updated_file, fieldnames=fieldnames)
        csv_writer.writeheader()
        csv_writer.writerows(rows)

for row_index, row in enumerate(rows):
    if processed_rows >= num_rows_to_process:
        break

    if row.get('Processed') == 'yes':
        print(f"Fila {row_index + 1} ya está completa. Pasando a la siguiente fila.")
        continue

    if any([not row.get(col, '').strip() for col in required_columns if col != 'Processed']):
        company_name = row.get('Company', '')
        company_url = row.get('Company URL', '')

        print(f"Procesando fila {row_index + 1} - Empresa: {company_name}")

        try:
            driver.get(company_url)
            time.sleep(2)

            div_xpath = '/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div/div/div/div[1]/div[2]'
            links = {
                'Website': row.get('Website', 'No tiene Website'),
                'LinkeIn': row.get('LinkeIn', 'No tiene LinkeIn'),
                'Faceboock': row.get('Faceboock', 'No tiene Faceboock'),
                'Twitter': row.get('Twitter', 'No tiene Twitter'),
                'Descripción': row.get('Descripción', 'No tiene descripción'),
            }

            if any([not row.get(col, '').strip() for col in required_columns if col != 'Processed']):
                try:
                    div_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, div_xpath))
                    )
                    link_elements = div_element.find_elements(By.TAG_NAME, 'a')
                    
                    for link in link_elements:
                        href = link.get_attribute('href')
                        if href:
                            if 'linkedin.com' in href:
                                links['LinkeIn'] = href
                            elif 'facebook.com' in href:
                                links['Faceboock'] = href
                            elif 'twitter.com' in href or 'x.com' in href:
                                links['Twitter'] = href
                            else:
                                if links['Website'] == 'No tiene Website':
                                    links['Website'] = href
                
                except Exception as e:
                    print(f"No se pudo encontrar el div con enlaces: {e}")

                show_more_xpath = '/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div[1]/div[1]/div[2]/div/div/div/div/div/div[1]/div/div/div/span/a'
                try:
                    show_more_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, show_more_xpath))
                    )
                    show_more_button.click()
                    
                    time.sleep(2)
                    
                    description_xpath = '/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div[1]/div[1]/div[2]/div/div/div/div/div/div[1]/div/div/div/span/span'
                    try:
                        description_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, description_xpath))
                        )
                        description = description_element.text
                        if description:
                            links['Descripción'] = description
                        else:
                            links['Descripción'] = 'No tiene descripción'
                        
                    except Exception as e:
                        links['Descripción'] = 'No tiene descripción'
                        
                except Exception as e:
                    links['Descripción'] = 'No tiene descripción'

            rows[row_index].update(links)
            rows[row_index]['Processed'] = 'yes'
            processed_rows += 1

            # Escribir los cambios en el archivo CSV
            write_to_csv(rows, fieldnames)

        except Exception as e:
            print(f"Error al procesar la URL {company_url}: {e}")
            rows[row_index].update({
                'Website': 'Error de página',
                'LinkeIn': 'Error de página',
                'Faceboock': 'Error de página',
                'Twitter': 'Error de página',
                'Descripción': 'Error de página',
                'Processed': 'yes'
            })
            processed_rows += 1
            # Escribir los cambios en el archivo CSV
            write_to_csv(rows, fieldnames)
            # Cerrar el navegador y reiniciar
            driver.quit()
            driver = start_driver()

driver.quit()
print(f"{csv_file_path} actualizado correctamente")

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import os

# Ruta absoluta del directorio de datos del usuario (debe coincidir con la usada en el primer script)
user_data_dir = os.path.abspath("chrome_user_data")

# Configurar opciones de Chrome
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Abrir el navegador en pantalla completa
chrome_options.add_argument(f"user-data-dir={user_data_dir}")  # Reusar la sesión iniciada

# Descargar y configurar el controlador de Chrome automáticamente
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Leer el archivo CSV, omitiendo líneas problemáticas
input_file = 'Resultados/Empresas/Apollo_Acounts_1-10 copy.csv'
df = pd.read_csv(input_file, on_bad_lines='skip')

# Verificar si el archivo final existe, si no, crear un DataFrame vacío
output_file = 'Prospects_with_all_data.csv'
if os.path.exists(output_file):
    final_df = pd.read_csv(output_file)
else:
    final_df = pd.DataFrame()

def extract_table_data(table_xpath, pagination_xpath):
    table_data = []
    while True:
        # Buscar la tabla con el XPath especificado
        table = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, table_xpath))
        )

        # Extraer los datos de la tabla
        time.sleep(2)
        table_rows = table.find_elements(By.TAG_NAME, 'tr')

        if not table_rows:
            break

        headers = [header.text for header in table_rows[0].find_elements(By.TAG_NAME, 'th')]
        if not headers:  # Si no hay elementos <th>, usar <td> de la primera fila
            headers = [header.text for header in table_rows[0].find_elements(By.TAG_NAME, 'td')]

        try:
            quick_actions_index = headers.index('Quick Actions')
        except ValueError:
            quick_actions_index = -1

        if quick_actions_index != -1:
            headers.pop(quick_actions_index)

        for row in table_rows[1:]:  # Omitir la primera fila de encabezados
            cells = row.find_elements(By.TAG_NAME, 'td')
            row_data = [cell.text for i, cell in enumerate(cells) if i != quick_actions_index]

            if 'Name' in headers:
                name_cell = cells[headers.index('Name')]
                name_link_element = name_cell.find_element(By.TAG_NAME, 'a')
                apollo_profile_url = name_link_element.get_attribute('href')
                row_data.append(apollo_profile_url)
            else:
                row_data.append(None)

            table_data.append(row_data)

        headers.append('Apollo Profile URL')
        table_df = pd.DataFrame(table_data, columns=headers)

        if not table_df.empty:
            table_df['Company'] = company_name
            table_df['Website'] = website

            global final_df
            if not final_df.empty:
                final_df = pd.concat([final_df, table_df], ignore_index=True)
            else:
                final_df = table_df

        # Buscar el botón "Siguiente"
        try:
            next_button = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, pagination_xpath))
            )
            
            # Verificar si el botón "Siguiente" está habilitado
            if 'disabled' in next_button.get_attribute('class') or 'disabled' in next_button.get_attribute('aria-disabled'):
                break  # Si el botón está deshabilitado, salir del bucle
            
            next_button.click()
            time.sleep(2)
        except Exception:
            break  # Si no se encuentra el botón "Siguiente", salir del bucle

def process_company_row(row):
    global company_name, website
    company_url = row['Company URL']
    company_name = row['Company']
    website = row['Website']

    try:
        driver.get(company_url)

        try:
            # Intentar buscar el enlace 'Employees'
            employees_link = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div[2]/div/div[1]/div/a[5]'))
            )
            employees_link.click()
            extract_table_data('/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div[2]/div/div[2]/div/div/div/div/div/div[4]/div/div/div/div/div[2]/div/table',
                               '/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div[2]/div/div[2]/div/div/div/div/div/div[4]/div/div/div/div/div[3]/div/div[2]/button[2]')

        except Exception:
            # Si no se encuentra 'Employees', buscar 'People'
            people_link = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div[1]/div[2]/div/div/div[1]/div/a[3]'))
            )
            people_link.click()
            extract_table_data('/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div[1]/div[2]/div/div/div[2]/div/div/div/div/div/div[3]/div/div/div/div/div[2]/div/table',
                               '/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div[1]/div[2]/div/div/div[2]/div/div/div/div/div/div[3]/div/div/div/div/div[3]/div/div[2]/button[2]')
        
        final_df.to_csv(output_file, index=False)
        # print(f"Datos de '{company_name}' añadidos a '{output_file}'.")

        # Marcar fila como procesada en el archivo original
        df.at[row.name, 'Processed'] = 'Yes'
        df.to_csv(input_file, index=False)
        # print(f"Archivo original actualizado: '{company_name}' marcado como procesado.")

    except Exception as e:
        print(f"No se pudo completar la tarea para {company_name}: {str(e)}")

# Iterar sobre cada fila del DataFrame, si no está procesada
for index, row in df.iterrows():
    if pd.isna(row.get('Processed')) or row['Processed'] != 'Yes':
        print(f"Procesando fila {index + 1} de {len(df)}: {row['Company']}")  # Mensaje para imprimir la fila actual
        process_company_row(row)

print("Proceso terminado")

# Cerrar el navegador
driver.quit()


Procesando fila 603 de 24070: Ultraview AI
No se pudo completar la tarea para Ultraview AI: Message: 
Stacktrace:
	GetHandleVerifier [0x00728923+23283]
	(No symbol) [0x006EE934]
	(No symbol) [0x00620733]
	(No symbol) [0x0066326F]
	(No symbol) [0x006634AB]
	(No symbol) [0x0069EE42]
	(No symbol) [0x00684464]
	(No symbol) [0x0069CB8D]
	(No symbol) [0x006841B6]
	(No symbol) [0x00658017]
	(No symbol) [0x0065890D]
	GetHandleVerifier [0x0081A5F3+1013699]
	GetHandleVerifier [0x00823E4C+1052700]
	GetHandleVerifier [0x0081D4B4+1025668]
	GetHandleVerifier [0x0074EA2B+179195]
	(No symbol) [0x006F6833]
	(No symbol) [0x006F3198]
	(No symbol) [0x006F3337]
	(No symbol) [0x006EB4BE]
	BaseThreadInitThunk [0x760DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x770280CE+286]
	RtlGetAppContainerNamedObjectPath [0x7702809E+238]

Procesando fila 1110 de 24070: Denver Basic Income Project
No se pudo completar la tarea para Denver Basic Income Project: Message: 
Stacktrace:
	GetHandleVerifier [0x00728923+23283]
